In [6]:
import pandas as pd
import numpy as np
import re
import glob
import os
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import LSTM, Dense

In [7]:
path = "../stocks_1m/raw_data_candles/*.csv" # for the 1 min data
path2 = "../stocks_1d/raw_data_candles/*.csv" # # for the 1 day data

# This block creates a DF for each stock CSV file in the folder (path)

In [8]:

dfs = []
dfs_names = []
for file in glob.glob(path):
    # extract the file name without the extension
    file_name = os.path.splitext(os.path.basename(file))[0]
    dfs_names.append(file_name)
    # read the CSV file into a dataframe
    df = pd.read_csv(file)
    
    
    #preprocess:
    df = df.drop(columns="symbol")
    df["time"] = pd.to_datetime(df["time"])
    df.index = df.pop('time')
    
    # creat list:
    dfs.append(df)
    


# This is the step where we can join the tweet data separated by stock

# This converts the data into the input shape necessary for  the LSTM, and splits X and y

In [34]:
nr_dataframes = len(dfs)
nr_rows = 360
nr_features = 5

# Create an empty array to store the data
x_array = np.zeros((nr_dataframes, nr_rows, nr_features))
y_array = np.zeros((nr_dataframes, nr_rows))

# Loop through the dataframes and slice the last 1000 rows
for i, df in enumerate(dfs):
    sliced_df_x = df[:-1].tail(nr_rows)
    sliced_df_y = df.tail(nr_rows)

    # Convert the sliced dataframe to a numpy array
    x_df = np.array(sliced_df_x)
    

    # Add the numpy array to the empty array
    x_array[i, :, :] = x_df

    # Extract the 'close' column as a numpy array and add it to y_array
    close_prices = sliced_df_y['close'].to_numpy()
    y_array[i, :] = close_prices
    
    X = x_array
    y = np.expand_dims(y_array.astype(np.float32),axis=-1)

# Print the shape of the final arrays
print(X.shape)
print(y.shape)

(29, 360, 5)
(29, 360, 1)


In [35]:
X

array([[[8.79900000e+01, 8.79900000e+01, 8.78900000e+01, 8.79658000e+01,
         9.86990000e+04],
        [8.79500000e+01, 8.79600000e+01, 8.78800000e+01, 8.79200000e+01,
         1.09935000e+05],
        [8.79250000e+01, 8.80700000e+01, 8.79250000e+01, 8.80415000e+01,
         1.08495000e+05],
        ...,
        [8.59600000e+01, 8.59600000e+01, 8.59600000e+01, 8.59600000e+01,
         1.19000000e+02],
        [8.59600000e+01, 8.59600000e+01, 8.59600000e+01, 8.59600000e+01,
         2.36000000e+02],
        [8.60000000e+01, 8.60000000e+01, 8.60000000e+01, 8.60000000e+01,
         5.66000000e+02]],

       [[2.62477277e+02, 2.62696723e+02, 2.62467302e+02, 2.62547200e+02,
         2.12110000e+04],
        [2.62587000e+02, 2.62636874e+02, 2.62477277e+02, 2.62507201e+02,
         3.10070000e+04],
        [2.62507201e+02, 2.62936119e+02, 2.62427403e+02, 2.62886244e+02,
         5.98560000e+04],
        ...,
        [2.57360197e+02, 2.57360197e+02, 2.57350222e+02, 2.57350222e+02,
        

# Next step is scalling tha data and feed it to the model (review)

In [13]:
scaler = MinMaxScaler(feature_range=(0, 1))


In [35]:
model = Sequential()
model.add(LSTM(units=50, return_sequences=True, input_shape=(n_steps, n_features)))
model.add(LSTM(units=50))
model.add(Dense(units=1))
model.compile(optimizer='adam', loss='mean_squared_error')

In [36]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_4 (LSTM)               (None, 360, 50)           11200     
                                                                 
 lstm_5 (LSTM)               (None, 50)                20200     
                                                                 
 dense_2 (Dense)             (None, 1)                 51        
                                                                 
Total params: 31,451
Trainable params: 31,451
Non-trainable params: 0
_________________________________________________________________


In [37]:
model.fit(train_X, train_y, epochs=10, batch_size=32)

Epoch 1/10


ValueError: in user code:

    File "/home/armando/.pyenv/versions/3.10.6/envs/lewagon/lib/python3.10/site-packages/keras/engine/training.py", line 1160, in train_function  *
        return step_function(self, iterator)
    File "/home/armando/.pyenv/versions/3.10.6/envs/lewagon/lib/python3.10/site-packages/keras/engine/training.py", line 1146, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/home/armando/.pyenv/versions/3.10.6/envs/lewagon/lib/python3.10/site-packages/keras/engine/training.py", line 1135, in run_step  **
        outputs = model.train_step(data)
    File "/home/armando/.pyenv/versions/3.10.6/envs/lewagon/lib/python3.10/site-packages/keras/engine/training.py", line 993, in train_step
        y_pred = self(x, training=True)
    File "/home/armando/.pyenv/versions/3.10.6/envs/lewagon/lib/python3.10/site-packages/keras/utils/traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from

In [ ]:
for df in df:
    df = df.drop("symbol")
    


In [ ]:
df.columns

In [ ]:
df.info()

In [ ]:
def clean_cashtags(st):
    letters = re.findall('[A-Za-z]+', st)
    return ' '.join(letters)

In [ ]:
df = df.dropna(subset=["cashtags"])

In [ ]:
df["cashtags"] = df["cashtags"].apply(clean_cashtags)

In [ ]:
def del_timestamp(st):
    return st.replace("+00:00", "")

In [ ]:
df["date"] = df["date"].apply(del_timestamp)

In [ ]:
df["date"] = pd.to_datetime(df['date'])

In [ ]:
df.index = df.pop('date')

In [ ]:
df[df["cashtags"] == "MSFT"]

In [ ]:
df_stocks = pd.read_csv("AAPL.csv")
df_stocks.columns

In [ ]:
df_stocks = df_stocks[['time', 'close']]
df_stocks.info()

In [ ]:
df_stocks["time"] = pd.to_datetime(df_stocks["time"])

In [ ]:
df_stocks.index = df_stocks.pop("time")

In [ ]:
df_stocks

In [ ]:
scaller = MinMaxScaler()

df_stocks["close_scalled"] = scaller.fit_transform(df_stocks[["close"]])


In [ ]:
df_stocks

In [ ]:
df

In [ ]:
merged_df = pd.merge(df, df_stocks, left_index=True, right_index=True)

In [ ]:
merged_df

In [ ]:
merged_short = merged_df[['tweet', 'username', 'likeCount', 'replyCount',
       'retweetCount', 'sentiment', 'close']]

In [ ]:
merged_short

In [ ]:
df_stocks